In [1]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
from sklearn.utils import shuffle

import sys 
sys.path.append('../src/utils/')
sys.path.append('../src/models/')
sys.path.append('../src/visualization/')

from particles import Photon
from seed_finder import SeedFinder

%load_ext autoreload
%autoreload 2

# Load and prepare data for training from one- and two-particle samples

In [2]:
photon_1 = Photon(n_pcl=1)
photon_2 = Photon(n_pcl=2)

photons = [photon_1, photon_2]

In [3]:
Xtrain, ytrain = [], []
Xvalid, yvalid = [], []

for photon in photons:
    Xi, yi = photon.data_for_seed_finder("valid")
    Xtrain.extend(Xi)
    ytrain.extend(yi)

    Xi, yi = photon.data_for_seed_finder("test")
    Xvalid.extend(Xi)
    yvalid.extend(yi)
Xtrain, ytrain = np.array(Xtrain), np.array(ytrain)
Xvalid, yvalid = np.array(Xvalid), np.array(yvalid)

100%|██████████| 200000/200000 [00:55<00:00, 3624.32it/s]


(1877250,) (1877250, 7, 7, 1)


100%|██████████| 100000/100000 [00:19<00:00, 5251.17it/s]


(937190,) (937190, 7, 7, 1)


100%|██████████| 100000/100000 [00:28<00:00, 3509.01it/s]


(1481032,) (1481032, 7, 7, 1)


100%|██████████| 50000/50000 [00:12<00:00, 3968.10it/s]


(740027,) (740027, 7, 7, 1)


In [4]:
# shuffle the samples randomly
Xtrain, ytrain = shuffle(Xtrain, ytrain, random_state=42)
Xvalid, yvalid = shuffle(Xvalid, yvalid, random_state=42)

# Prepare the model and training

In [5]:
model = SeedFinder().architecture()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss=tf.keras.losses.BinaryCrossentropy())
model.summary()

2024-01-09 17:36:33.526234: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-01-09 17:36:33.526697: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-01-09 17:36:33.526985: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-01-09 17:36:33.527445: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-09 17:36:33.528071: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 7, 7, 1)]         0         
                                                                 
 conv2d (Conv2D)             (None, 5, 5, 128)         1280      
                                                                 
 conv2d_1 (Conv2D)           (None, 3, 3, 256)         295168    
                                                                 
 flatten (Flatten)           (None, 2304)              0         
                                                                 
 dense (Dense)               (None, 2400)              5532000   
                                                                 
 dense_1 (Dense)             (None, 500)               1200500   
                                                                 
 dense_2 (Dense)             (None, 1)                 501   

In [6]:
filepath = 'models/seed_finder/run_1/'

checkpoint = ModelCheckpoint(
        str(filepath)+"/weights.{epoch:02d}-{val_loss:.4f}.hdf5", 
        monitor='val_loss', verbose = 1, save_best_only = True, 
        mode='auto', save_freq='epoch')

callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 30)

In [7]:
history = model.fit(Xtrain, ytrain,
                    validation_data=(Xvalid, yvalid), 
                    epochs=100, batch_size=64, 
                    callbacks=[checkpoint, callback])

Epoch 1/100


2024-01-09 17:36:36.374839: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-01-09 17:36:36.416114: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


52474/52474 [==============================] - ETA: 0s - loss: 0.0251
Epoch 1: val_loss improved from inf to 0.02112, saving model to models/seed_finder/run_1/weights.01-0.0211.hdf5
52474/52474 [==============================] - 948s 18ms/step - loss: 0.0251 - val_loss: 0.0211
Epoch 2/100
    1/52474 [..............................] - ETA: 25:52 - loss: 0.0046

/Users/polinasimkina/miniconda3/envs/envdeep/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


52474/52474 [==============================] - ETA: 0s - loss: 0.0181
Epoch 2: val_loss improved from 0.02112 to 0.01613, saving model to models/seed_finder/run_1/weights.02-0.0161.hdf5
52474/52474 [==============================] - 942s 18ms/step - loss: 0.0181 - val_loss: 0.0161
Epoch 3/100
52474/52474 [==============================] - ETA: 0s - loss: 0.0161
Epoch 3: val_loss improved from 0.01613 to 0.01493, saving model to models/seed_finder/run_1/weights.03-0.0149.hdf5
52474/52474 [==============================] - 960s 18ms/step - loss: 0.0161 - val_loss: 0.0149
Epoch 4/100
52474/52474 [==============================] - ETA: 0s - loss: 0.0149
Epoch 4: val_loss improved from 0.01493 to 0.01386, saving model to models/seed_finder/run_1/weights.04-0.0139.hdf5
52474/52474 [==============================] - 970s 18ms/step - loss: 0.0149 - val_loss: 0.0139
Epoch 5/100
52474/52474 [==============================] - ETA: 0s - loss: 0.0140
Epoch 5: val_loss improved from 0.01386 to 0.013

In [ ]:
model.save(str(filepath))

# Draw loss function 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['figure.figsize'] = (5,5)
plt.rcParams['figure.dpi'] = 150

plt.scatter(np.arange(0,100), history.history["loss"], s=1., label='loss')
plt.scatter(np.arange(0,100), history.history["val_loss"], s=1., label='val_loss')
plt.legend()
plt.savefig(str(filepath) + '/loss_function')
#plt.xlim(0,500)
#plt.ylim(0.,0.01)